In [1]:
def get_parameters(filename):
    with open(filename, "r") as f:
        vehicle_size = [int(size) for size in f.readline().replace("\n", "").split(" ")]
        nb_items = int(f.readline().replace("\n", ""))
        items_size = []
        for line in f.readlines():
            items_size.append([int(size) for size in line.replace("\n", "").split(" ")])
    
    assert nb_items == len(items_size)
    return vehicle_size, nb_items, items_size

truck_dimensions, _, item_dimensions = get_parameters("input")
truck_dimensions, item_dimensions

([320, 170, 170],
 [[50, 10, 190],
  [190, 190, 20],
  [110, 110, 150],
  [160, 40, 140],
  [90, 80, 120],
  [20, 150, 170],
  [10, 120, 160],
  [80, 30, 150],
  [90, 130, 140],
  [90, 60, 110],
  [90, 70, 40],
  [70, 80, 100],
  [70, 40, 30],
  [170, 190, 180],
  [30, 110, 60],
  [70, 140, 80],
  [40, 80, 160],
  [50, 150, 90],
  [160, 90, 160],
  [110, 140, 190],
  [140, 110, 70],
  [40, 30, 70],
  [80, 150, 170],
  [190, 10, 80],
  [170, 120, 90],
  [150, 30, 110]])

In [2]:
#truck_dimensions = [120, 330, 230]
#item_dimensions = [[80, 30, 230], [50, 160, 50], [200, 200, 10], [10, 10, 10], [150, 10, 10], [500, 500, 500]]
#nb_items = len(item_dimensions)

item_dimensions = item_dimensions[:14]

In [3]:
for item_size in item_dimensions:
    print(item_size[0] * item_size[1] * item_size[2])

95000
722000
1815000
896000
864000
510000
192000
360000
1638000
594000
252000
560000
84000
5814000


|✅|❌
|:-:|:-:
|Pieces|Position
|Overlap|Rotation
|Max Pieces|Truck size
||Max Items Volume|

In [4]:
import pulp

def solve_packing_problem(items, weight_num_items):
    model = pulp.LpProblem("Truck_packing_problem", pulp.LpMaximize)
    num_items = len(items)

    x = pulp.LpVariable.dicts("x", range(num_items), cat='Binary')

    total_items = sum(x[i] for i in range(num_items))
    model += weight_num_items * total_items 

    model.solve()
    packed_items = [i for i in range(num_items) if pulp.value(x[i]) == 1]
    return packed_items

weight_num_items = 10

packed_items = solve_packing_problem(item_dimensions, weight_num_items)
print(packed_items)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]


|✅|❌
|:-:|:-:
|Pieces|Rotation
|Overlap|
|Max Pieces|
|Position|
|Truck size|
|Max Items Volume|

In [5]:
import pulp

def solve_packing_problem(truck_dims, items, weight_num_items, weight_volume):
    model = pulp.LpProblem("Truck_packing_problem", pulp.LpMaximize)
    num_items = len(items)

    x = pulp.LpVariable.dicts("x", range(num_items), cat='Binary')
    pos = pulp.LpVariable.dicts("pos", [(i, j) for i in range(num_items) for j in range(3)], 
                                lowBound=0, cat='Continuous')

    total_volume = sum(items[i][0] * items[i][1] * items[i][2] * x[i] for i in range(num_items))
    total_items = sum(x[i] for i in range(num_items))
    model += weight_num_items * total_items + weight_volume * total_volume

    for i in range(num_items):
        for j in range(3):
            model += pos[(i, j)] + items[i][j] * x[i] <= truck_dims[j]

    for i in range(num_items):
        for j in range(i + 1, num_items):
            model += x[i] + x[j] <= 1

    model.solve()
    packed_items = [i for i in range(num_items) if pulp.value(x[i]) == 1]
    positions = {i: (pulp.value(pos[(i, 0)]), pulp.value(pos[(i, 1)]), pulp.value(pos[(i, 2)])) 
                 for i in packed_items}
    return packed_items, positions

weight_num_items = 2
weight_volume = 10

packed_items, positions = solve_packing_problem(truck_dimensions, item_dimensions, weight_num_items, weight_volume)
print("Items packed:", packed_items)
print("Positions:", positions)

Items packed: [2]
Positions: {2: (0.0, 0.0, 0.0)}


|✅|❌
|:-:|:-:
|Pieces|
|Overlap|
|Max Pieces|
|Position|
|Truck size|
|Max Items Volume|
|Rotation|

In [6]:
import pulp

def solve_packing_problem(truck_dims, items, weight_num_items, weight_volume):
    model: pulp.LpProblem = pulp.LpProblem("Truck_packing_problem", pulp.LpMaximize)
    num_items = len(items)

    x = pulp.LpVariable.dicts("x", range(num_items), cat='Binary')
    pos = pulp.LpVariable.dicts("pos", [(i, j) for i in range(num_items) for j in range(3)], 
                                lowBound=0, cat='Continuous')
    orientation = pulp.LpVariable.dicts("orientation", 
                                        [(i, j) for i in range(num_items) for j in range(6)], 
                                        cat='Binary')

    total_volume = sum(items[i][0] * items[i][1] * items[i][2] * x[i] for i in range(num_items))
    total_items = sum(x[i] for i in range(num_items))
    model += weight_num_items * total_items + weight_volume * total_volume

    for i in range(num_items):
        model += sum(orientation[(i, j)] for j in range(6)) == x[i]
        for j in range(3):
            real_dim = sum(orientation[(i, k)] * items[i][k % 3] for k in range(6))
            model += pos[(i, j)] + real_dim <= truck_dims[j]

    for i in range(num_items):
        real_dim_i_x = sum(orientation[(i, k)] * items[i][k % 3] for k in range(6))
        real_dim_i_y = sum(orientation[(i, k)] * items[i][(k + 1) % 3] for k in range(6))
        real_dim_i_z = sum(orientation[(i, k)] * items[i][(k + 2) % 3] for k in range(6))

        # Assurer que chaque pièce rentre dans le camion
        model += pos[(i, 0)] + real_dim_i_x <= truck_dims[0]
        model += pos[(i, 1)] + real_dim_i_y <= truck_dims[1]
        model += pos[(i, 2)] + real_dim_i_z <= truck_dims[2]

        # Contraintes de non-chevauchement
        for j in range(i + 1, num_items):
            # Calculer les dimensions réelles de l'item j
            real_dim_j_x = sum(orientation[(j, k)] * items[j][k % 3] for k in range(6))
            real_dim_j_y = sum(orientation[(j, k)] * items[j][(k + 1) % 3] for k in range(6))
            real_dim_j_z = sum(orientation[(j, k)] * items[j][(k + 2) % 3] for k in range(6))

            left = pulp.LpVariable(f"left_{i}_{j}", cat='Binary')
            right = pulp.LpVariable(f"right_{i}_{j}", cat='Binary')
            above = pulp.LpVariable(f"above_{i}_{j}", cat='Binary')
            below = pulp.LpVariable(f"below_{i}_{j}", cat='Binary')
            front = pulp.LpVariable(f"front_{i}_{j}", cat='Binary')
            back = pulp.LpVariable(f"back_{i}_{j}", cat='Binary')

            model += pos[(i, 0)] + real_dim_i_x <= pos[(j, 0)] + truck_dims[0]*(1 - left)
            model += pos[(j, 0)] + real_dim_j_x <= pos[(i, 0)] + truck_dims[0]*(1 - right)
            model += pos[(i, 1)] + real_dim_i_y <= pos[(j, 1)] + truck_dims[1]*(1 - below)
            model += pos[(j, 1)] + real_dim_j_y <= pos[(i, 1)] + truck_dims[1]*(1 - above)
            model += pos[(i, 2)] + real_dim_i_z <= pos[(j, 2)] + truck_dims[2]*(1 - back)
            model += pos[(j, 2)] + real_dim_j_z <= pos[(i, 2)] + truck_dims[2]*(1 - front)

            model += left + right + above + below + front + back >= x[i] + x[j] - 1


    model.solve()
    packed_items = [i for i in range(num_items) if pulp.value(x[i]) == 1]
    positions = {i: (pulp.value(pos[(i, 0)]), pulp.value(pos[(i, 1)]), pulp.value(pos[(i, 2)])) 
                 for i in packed_items}
    orientations = {i: [pulp.value(orientation[(i, j)]) for j in range(6)] 
                    for i in packed_items}
    return packed_items, positions, orientations

weight_num_items = 1
weight_volume = 5

packed_items, positions, orientations = solve_packing_problem(truck_dimensions, item_dimensions, weight_num_items, weight_volume)
packed_items, positions, orientations

([2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
 {2: (80.0, 60.0, 20.0),
  3: (280.0, 0.0, 0.0),
  4: (0.0, 50.0, 80.0),
  5: (120.0, 0.0, 0.0),
  6: (0.0, 0.0, 0.0),
  7: (0.0, 0.0, 20.0),
  8: (190.0, 40.0, 20.0),
  9: (80.0, 0.0, 20.0),
  10: (210.0, 0.0, 80.0),
  11: (0.0, 70.0, 10.0),
  12: (210.0, 0.0, 20.0)},
 {2: [1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
  3: [0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
  4: [0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
  5: [0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
  6: [0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
  7: [0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
  8: [0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
  9: [0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
  10: [0.0, 0.0, 0.0, 0.0, 1.0, 0.0],
  11: [0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
  12: [0.0, 0.0, 0.0, 1.0, 0.0, 0.0]})

In [7]:
def convert_to_output_format(packed_items, positions, orientations, items, file_path):
    with open(file_path, 'w') as file:
        file.write("SAT\n")
        for i in packed_items:
            x1, y1, z1 = positions[i]
            dim_x = sum(orientations[i][k] * items[i][k % 3] for k in range(6))
            dim_y = sum(orientations[i][k] * items[i][(k + 1) % 3] for k in range(6))
            dim_z = sum(orientations[i][k] * items[i][(k + 2) % 3] for k in range(6))
            x2, y2, z2 = x1 + dim_x, y1 + dim_y, z1 + dim_z

            file.write(f"0 {int(x1)} {int(y1)} {int(z1)} {int(x2)} {int(y2)} {int(z2)}\n")

file_path = 'output.txt'

convert_to_output_format(packed_items, positions, orientations, item_dimensions, file_path)

In [8]:
f"python visualize.py output.txt --truck-dimensions \"{'x'.join([str(dim) for dim in truck_dimensions])}\""

'python visualize.py output.txt --truck-dimensions "320x170x170"'